## Visualizing the Representations:
I have trained a ResNet50 based architecture to learn meaningful representations from our data [here](https://www.kaggle.com/saimanojakondi/simsaim/notebook) using SIMSAIM. This notebook uses T-SNE to extract important features and visualize the embeddings.

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
import cv2
import numpy as np
import time
import random
from tqdm import tqdm as tqdm
import pandas as pd
import os


from efficientnet_pytorch import EfficientNet

from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
from albumentations.pytorch import ToTensorV2
from torchvision import models

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

In [ ]:
cfg = {
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'epochs':25,
    'batch_size':42,
    'lr':0.0001,
    'input_size':256,
    
}

backbone = models.resnet50(pretrained=True)
modules = list(backbone.children())[:-2]
backbone = nn.Sequential(*modules)
print(backbone)


In [ ]:
class CLASSIFIER(nn.Module):
    def __init__(self):
        super().__init__()
        self.effn = backbone
        self.average = nn.AvgPool2d((8,8))
        self.flatten = nn.Flatten()
    def forward(self,x):
        x = self.effn(x)
        x = self.average(x)
        x = self.flatten(x)
        return x
    
class MODEL(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = CLASSIFIER()
        if cfg['device']=='cuda':
            self.backbone.load_state_dict(torch.load('../input/simsaim-weights/I_am_trained_14.pt'))
        else:
            self.backbone.load_state_dict(torch.load('../input/simsaim-weights/I_am_trained_14.pt',map_location='cpu'))
        self.dense = nn.Linear(2048,5)
    def forward(self,x):
        x = self.backbone(x)
        x = self.dense(x)
        return x

model = MODEL()
x = torch.randn((1,3,256,256))
y = model(x)
print(y.size())

In [ ]:
class CASSAVA(Dataset):
    def __init__(self,
                 imagenames,
                 csv,
                 root_dir,
                 input_size=cfg['input_size'],
                 transforms=None,
                 train=True):
        self.imagenames = imagenames
        self.csv = csv
        self.root_dir = root_dir
        self.input_size = input_size
        self.transforms = transforms
        self.train = train
    def __len__(self):
        return len(imagenames)
    def get_onehot(self,label):
        onehot = np.zeros(5)
        onehot[label] = 1
        return onehot
    def __getitem__(self,idx):
        imagename = self.imagenames[idx]
        label = self.csv[self.csv['image_id']==imagename]['label']
        label = self.get_onehot(label)
        image = cv2.imread(self.root_dir+'/'+imagename)
        image = cv2.resize(image,(self.input_size,self.input_size))
        if self.transforms is not None:
            image_aug = self.transforms(image=image)['image']
        else:
            image_aug = image
        image_aug = normalize_and_to_tensor(image_aug)
        label = torch.from_numpy(label)
        return image_aug,label

In [ ]:
train_dir= '../input/cassava-leaf-disease-classification/train_images'
test_dir = '../input/cassava-leaf-disease-classification/test_images'
imagenames = [name for name in os.listdir(train_dir)]
csv = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
print(csv.head(10))
print(csv['label'].unique())

In [ ]:
t_dataset = CASSAVA(imagenames,csv,train_dir,transforms = None)
train_loader = DataLoader(dataset=t_dataset, batch_size=cfg['batch_size'], shuffle=True, num_workers=0)

In [ ]:
def normalize_and_to_tensor(img):
    transform = Compose([Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                       ToTensorV2(p=1.0)],p=1.0)
    return transform(image=img)['image']
total_features = []
y = []
model.to(cfg['device'])
for X,Y in tqdm(train_loader):
    X = X.to(cfg['device'])
    y = y + list(np.argmax(Y.numpy(),axis=-1))
    features = model(X)
    #print(features.size())
    features = features/(features.norm(dim=-1)[:,None]+1e-6)
    features = features.detach().cpu().numpy()
    features = list(features)
    total_features+=features

In [ ]:
total_features = np.asarray(total_features)
print(total_features.shape)

In [ ]:
print(len(y))

In [ ]:
from sklearn.manifold import TSNE
X_embedded = TSNE(n_components=2,n_iter=1000,verbose=1).fit_transform(total_features)

In [ ]:
color_dict = dict({0:'brown',
                  1:'green',
                  2: 'orange',
                  3: 'black',
                   4: 'dodgerblue'})
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
df_subset = {}
y = np.asarray(y)
df_subset['tsne-2d-one'] = X_embedded[:,0]
df_subset['tsne-2d-two'] = X_embedded[:,1]
df_subset['y'] = y
df_subset = pd.DataFrame.from_dict(df_subset)
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue='y',
    data=df_subset,
    legend="full",
    palette=color_dict,
    alpha=0.3
)

In [ ]:
'''import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
df = {}
y = np.asarray(y)
df['tsne-2d-one'] = X_embedded[:,0]
df['tsne-2d-two'] = X_embedded[:,1]
df['tsne-2d-three'] = X_embedded[:,2]
df['y'] = y
ax = plt.figure(figsize=(16,10)).gca(projection='3d')
ax.scatter(
    xs=df["tsne-2d-one"], 
    ys=df["tsne-2d-two"], 
    zs=df["tsne-2d-three"], 
    c=df["y"], 
    cmap='tab10'
)
ax.set_xlabel('tsne-2d-one')
ax.set_ylabel('tsne-2d-two')
ax.set_zlabel('tsne-2d-three')
plt.show()'''